In [ ]:
import pandas as pd
import numpy as np

In [2]:
df_cat = pd.read_excel('TRAIN_CATEGORICAL_METADATA_new.xlsx')
df_fun = pd.read_csv('TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
df_quant = pd.read_excel('TRAIN_QUANTITATIVE_METADATA_new.xlsx')
df_sol = pd.read_excel('TRAINING_SOLUTIONS.xlsx')

In [3]:
# need to check out missing values
print(df_cat.isnull().sum())
print(df_fun.isnull().sum())
print(df_quant.isnull().sum())
print(df_sol.isnull().sum())

participant_id                        0
Basic_Demos_Enroll_Year               0
Basic_Demos_Study_Site                0
PreInt_Demos_Fam_Child_Ethnicity     43
PreInt_Demos_Fam_Child_Race          54
MRI_Track_Scan_Location               3
Barratt_Barratt_P1_Edu               15
Barratt_Barratt_P1_Occ               31
Barratt_Barratt_P2_Edu              198
Barratt_Barratt_P2_Occ              222
dtype: int64
participant_id          0
0throw_1thcolumn        0
0throw_2thcolumn        0
0throw_3thcolumn        0
0throw_4thcolumn        0
                       ..
196throw_198thcolumn    0
196throw_199thcolumn    0
197throw_198thcolumn    0
197throw_199thcolumn    0
198throw_199thcolumn    0
Length: 19901, dtype: int64
participant_id                  0
EHQ_EHQ_Total                  13
ColorVision_CV_Score           23
APQ_P_APQ_P_CP                 12
APQ_P_APQ_P_ID                 12
APQ_P_APQ_P_INV                12
APQ_P_APQ_P_OPD                12
APQ_P_APQ_P_PM                 12
A

In [4]:
# categorical dataset
# fill missing data with mode
categorical_cols = df_cat.select_dtypes(include=['float64']).columns
df_cat[categorical_cols] = df_cat[categorical_cols].fillna(df_cat[categorical_cols].mode().iloc[0])

# functional dataset
# fill missing data with 0
df_fun = df_fun.fillna(0)

# quantitative dataset
# fill missing data with median
numeric_cols = df_quant.select_dtypes(include=[np.number]).columns
df_quant[numeric_cols] = df_quant[numeric_cols].fillna(df_quant[numeric_cols].median())

In [5]:
# do one-hot encoding for categorical columns/variables
df_cat.iloc[:, 1:] = df_cat.iloc[:, 1:].astype(str)

df_encoded = pd.get_dummies(df_cat.iloc[:, 1:], drop_first=True)
df_final = pd.concat([df_cat.iloc[:, [0]], df_encoded], axis=1)

C:\Users\miche\AppData\Local\Temp\ipykernel_19252\2740244880.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       2019
1       2017
2       2017
3       2018
4       2018
        ... 
1208    2019
1209    2018
1210    2018
1211    2019
1212    2017
Name: Basic_Demos_Enroll_Year, Length: 1213, dtype: object' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_cat.iloc[:, 1:] = df_cat.iloc[:, 1:].astype(str)
C:\Users\miche\AppData\Local\Temp\ipykernel_19252\2740244880.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       4
1       1
2       1
3       1
4       1
       ..
1208    4
1209    1
1210    3
1211    4
1212    1
Name: Basic_Demos_Study_Site, Length: 1213, dtype: object' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_cat.iloc[:, 1:] = df_cat.

In [6]:
df_final.shape

(1213, 54)

In [7]:
merged_df = df_final.merge(df_fun, on="participant_id").merge(df_quant, on="participant_id").merge(df_sol, 
                                                                                                 on="participant_id")

In [8]:
merged_df_sub = merged_df.iloc[:, 1:]

In [9]:
X = merged_df_sub.iloc[:, :-2]
y = merged_df_sub.iloc[:, -2:]

In [10]:
print(X.shape)
print(y.shape)

(1213, 19971)
(1213, 2)


In [11]:
# need to cut down number of columns - most of the columns come from df_fun

# variance threshold
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
print(selector)

# Fit the selector but don't transform yet
selector.fit(X)

# Get mask of features to keep (True) or remove (False)
feature_mask = selector.get_support()

# Get list of column names that are kept
selected_columns = X.columns[feature_mask].tolist()

# Now create the reduced DataFrame using the original DataFrame and selected columns
reduced_df = X[selected_columns]

print(reduced_df.shape)

VarianceThreshold(threshold=0.01)
(1213, 19958)


In [12]:
# correlation analysis
corr_matrix = reduced_df.corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.3)]

df_reduced = reduced_df.drop(columns=to_drop)
print(df_reduced.shape)

(1213, 71)


In [13]:
df_reduced

,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,PreInt_Demos_Fam_Child_Ethnicity_1.0,PreInt_Demos_Fam_Child_Ethnicity_2.0,PreInt_Demos_Fam_Child_Ethnicity_3.0,PreInt_Demos_Fam_Child_Race_1.0,PreInt_Demos_Fam_Child_Race_3.0,PreInt_Demos_Fam_Child_Race_8.0,PreInt_Demos_Fam_Child_Race_9.0,Barratt_Barratt_P1_Edu_15.0,...,52throw_143thcolumn,54throw_55thcolumn,55throw_143thcolumn,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM
0,False,False,True,False,False,False,False,False,False,False,...,0.211898,0.080683,0.667223,100.00,13.0,3.0,15.0,44.0,14.0,20.0
1,False,True,False,False,False,False,False,False,True,False,...,0.292354,0.437713,0.107788,92.27,14.0,3.0,12.0,35.0,25.0,28.0
2,False,True,True,False,False,False,False,False,False,False,...,0.161423,0.457295,0.256851,86.67,14.0,3.0,21.0,37.0,18.0,26.0
3,False,False,False,False,True,False,False,True,False,False,...,0.587227,0.395815,0.273971,93.34,14.0,3.0,11.0,42.0,15.0,20.0
4,False,False,False,False,False,True,False,False,False,False,...,0.186582,-0.029292,0.610008,0.00,14.0,8.0,12.0,35.0,22.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,False,False,True,False,False,True,False,False,False,False,...,0.177300,0.582539,0.289973,100.00,14.0,3.0,14.0,36.0,18.0,13.0
1209,False,False,False,False,False,False,False,False,False,False,...,0.203709,0.147176,-0.174189,46.67,14.0,3.0,18.0,42.0,21.0,15.0
1210,False,False,False,True,False,False,True,False,False,False,...,0.258452,0.378451,0.567541,-40.00,14.0,3.0,19.0,34.0,20.0,20.0
1211,False,False,False,False,False,True,False,False,False,False,...,0.156323,-0.001599,0.174046,100.00,14.0,3.0,9.0,45.0,23.0,11.0


In [16]:
# Concatenating side by side
df_combined = pd.concat([df_reduced, y], axis=1)

df_combined

,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,PreInt_Demos_Fam_Child_Ethnicity_1.0,PreInt_Demos_Fam_Child_Ethnicity_2.0,PreInt_Demos_Fam_Child_Ethnicity_3.0,PreInt_Demos_Fam_Child_Race_1.0,PreInt_Demos_Fam_Child_Race_3.0,PreInt_Demos_Fam_Child_Race_8.0,PreInt_Demos_Fam_Child_Race_9.0,Barratt_Barratt_P1_Edu_15.0,...,55throw_143thcolumn,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,ADHD_Outcome,Sex_F
0,False,False,True,False,False,False,False,False,False,False,...,0.667223,100.00,13.0,3.0,15.0,44.0,14.0,20.0,1,0
1,False,True,False,False,False,False,False,False,True,False,...,0.107788,92.27,14.0,3.0,12.0,35.0,25.0,28.0,1,0
2,False,True,True,False,False,False,False,False,False,False,...,0.256851,86.67,14.0,3.0,21.0,37.0,18.0,26.0,0,1
3,False,False,False,False,True,False,False,True,False,False,...,0.273971,93.34,14.0,3.0,11.0,42.0,15.0,20.0,0,1
4,False,False,False,False,False,True,False,False,False,False,...,0.610008,0.00,14.0,8.0,12.0,35.0,22.0,12.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,False,False,True,False,False,True,False,False,False,False,...,0.289973,100.00,14.0,3.0,14.0,36.0,18.0,13.0,0,1
1209,False,False,False,False,False,False,False,False,False,False,...,-0.174189,46.67,14.0,3.0,18.0,42.0,21.0,15.0,1,0
1210,False,False,False,True,False,False,True,False,False,False,...,0.567541,-40.00,14.0,3.0,19.0,34.0,20.0,20.0,1,1
1211,False,False,False,False,False,True,False,False,False,False,...,0.174046,100.00,14.0,3.0,9.0,45.0,23.0,11.0,1,0


In [18]:
df_combined.to_csv('nn_data.csv')

In [ ]:
# avoid using PCA because we want the columns to be interpretable
# from sklearn.decomposition import PCA

# pca = PCA(n_components=100)  # Choose based on explained variance
# reduced_features = pca.fit_transform(df_reduced)
# reduced_features

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier()
# model.fit(X, y)
# important_features = X.columns[np.argsort(model.feature_importances_)[-100:]]
# df_reduced = df[important_features]
